In [1]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 22.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import time
import faiss
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# -----------------------------
# Parameters
# -----------------------------
n_of_vectors = 10_000_000
vector_dimension = 64
nof_top_matches = 5
nlist = 100
nprobe = 10

In [14]:
# -----------------------------
# Data
# -----------------------------
vectors = np.random.random((n_of_vectors, vector_dimension)).astype('float32')
query_vector = np.random.random((1, vector_dimension)).astype('float32')

print("Dataset shape:", vectors.shape)


Dataset shape: (10000000, 64)


In [15]:
# -----------------------------
# 1. Brute Force with sklearn
# -----------------------------
def brute_force_search(vectors, query_vector, nof_top_matches=5):
    sims = cosine_similarity(query_vector, vectors).flatten()
    top_indices = sims.argsort()[::-1][:nof_top_matches]
    return top_indices

start = time.time()
tp = brute_force_search(vectors, query_vector, nof_top_matches)
end = time.time()
print("\n[sklearn cosine brute]")
print("Time: {:.4f} seconds".format(end - start))
print("Top indices:", tp[:10])


[sklearn cosine brute]
Time: 3.9434 seconds
Top indices: [2570254 6089359 1477026 3911324 9302068]


In [16]:
# -----------------------------
# 2. FAISS Flat Index (exact L2)
# -----------------------------
index_flat = faiss.IndexFlatL2(vector_dimension)  # brute-force L2
index_flat.add(vectors)

start = time.time()
D, I = index_flat.search(query_vector, nof_top_matches)
end = time.time()
print("\n[FAISS IndexFlatL2 brute]")
print("Time: {:.4f} seconds".format(end - start))
print("Top indices:", I[0][:10])


[FAISS IndexFlatL2 brute]
Time: 0.2737 seconds
Top indices: [2326964 2455147 7654379 4982754 3719398]


In [17]:
# -----------------------------
# 3. FAISS IVF-Flat (approximate)
# -----------------------------
quantizer = faiss.IndexFlatL2(vector_dimension)       # coarse quantizer
index_ivf = faiss.IndexIVFFlat(quantizer, vector_dimension, nlist, faiss.METRIC_L2)

# Train IVF on dataset
start = time.time()
index_ivf.train(vectors)      # learns cluster centroids
end = time.time()
print("\n[FAISS IndexIVFFlat] Training time: {:.4f} seconds".format(end - start))

# Add dataset to IVF
index_ivf.add(vectors)

# Set how many clusters to probe
index_ivf.nprobe = nprobe

# Search
start = time.time()
D_ivf, I_ivf = index_ivf.search(query_vector, nof_top_matches)
end = time.time()
print("[FAISS IndexIVFFlat] Query time: {:.4f} seconds".format(end - start))
print("Top indices:", I_ivf[0][:10])



[FAISS IndexIVFFlat] Training time: 1.3930 seconds
[FAISS IndexIVFFlat] Query time: 0.0309 seconds
Top indices: [2326964 4982754 4895345 7752522 9023717]


In [ ]:
# -----------------------------
# Summary
# -----------------------------
print("\n--- Summary ---")
print("sklearn brute cosine: slow, exact cosine")
print("FAISS FlatL2: fast brute-force, exact L2")
print("FAISS IVFFlat: approximate, tunable speed/accuracy with nlist/nprobe")


--- Summary ---
sklearn brute cosine: slow, exact cosine
FAISS FlatL2: fast brute-force, exact L2
FAISS IVFFlat: approximate, tunable speed/accuracy with nlist/nprobe
